# Bagging
É uma técnica de __ensemble__ usada para melhorar a precisão e reduzir a variância dos modelos, assim como busca tentar evitar ao máximo o overfitting.

 Seu principal objetivo é aumentar a robustez e estabilidade dos algoritmos que possuem alta variância, como as _árvores de decisão_, combinando muitas versões do mesmo modelo, mas treinados em diferentes subconjuntos de dados.

 __Como funciona:__
 - Primeiramente, é criado um subconjunto de dados (__Bootstrap__). A ideia é que sejam criados diferentes subconjuntos de dados de forma aleatória. Isso envolve a seleção desses dados de forma aleatória e __com reposição__, ou seja, uma mesma amostra pode ser escolhida mais de uma vez para diferentes subconjuntos.
 - É realizado o treinamento de múltiplos modelos onde, para cada subconjunto criado, é treinado um modelo completamente independente dos outros.
 - Após o treinamento desses modelos, é realizada a __agregação__. Onde as previsões são combinadas de forma a melhorar o desempenho do modelo final. Para problemas de regressão, geralmente é utilizada a média das previsões dos modelos. Para problemas de classificação, geralmente se usa a classe mais votada pelos modelos.


 __Principais objetivos:__
 - Reduzir a variância;
 - Evitar overfitting;
 - Melhorar a acurácia.

<br>

 É importante ressaltar que a técnica de Bagging pode ser utilizada em outros algoritmos de machine learning. Alguns exemplos onde o baggins pode ser aplicado são:
 - Árvores de decisão individuais, sem aplicar o conceito de florestas;
 - KNN;
 - Regressão Linear;
 - Redes Neurais;
 - SVM;
 - Entre outros.

In [1]:
# Importação do google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Importações
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

In [3]:
alvo = pd.read_pickle('/content/drive/MyDrive/Udemy/ML com Python/1 - Aprendizado Supervisionado: Classificacao/heart.pkl')

# Variáveis previsoras onde as variáveis categóricas foram transformadas em numéricas manualmente, sem escalonamento
previsores = pd.read_pickle('/content/drive/MyDrive/Udemy/ML com Python/1 - Aprendizado Supervisionado: Classificacao/heart2.pkl')

# previsores_esc = pd.read_pickle('/content/drive/MyDrive/Udemy/ML com Python/1 - Aprendizado Supervisionado: Classificacao/heart3.pkl')

# Variáveis previsoras onde as variáveis categóricas foram transformadas em numéricas pelo LabelEncoder.
previsores2 = pd.read_pickle('/content/drive/MyDrive/Udemy/ML com Python/1 - Aprendizado Supervisionado: Classificacao/heart4.pkl')

# Variáveis previsoras onde as variáveis categóricas foram transformadas em numéricas pelo LabelEncoder e OneHotEncoder, sem escalonamento.
previsores3 = pd.read_pickle('/content/drive/MyDrive/Udemy/ML com Python/1 - Aprendizado Supervisionado: Classificacao/heart5.pkl')

# Variáveis previsoras onde as variáveis categóricas foram transformadas pelo LabelEncoder e OHE, com escalonamento.
previsores3_esc = pd.read_pickle('/content/drive/MyDrive/Udemy/ML com Python/1 - Aprendizado Supervisionado: Classificacao/heart6.pkl')

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

In [5]:
# Criação do modelo
tree = DecisionTreeClassifier()
skfold = StratifiedKFold(n_splits=3)
resultado = cross_val_score(tree, previsores, alvo, cv=skfold)
print(resultado.mean())

0.74799099967856


Neste código acima utilizamos os valores padrão da árvore de decisão. Não realizamos nenhum ajuste ou otimização, e isso pode levar nosso modelo ao overfitting.

Vamos mexer um pouco nos parâmetros da árvore.

In [6]:
from sklearn.model_selection import GridSearchCV

In [7]:
# Valores a testar
min_splits = np.array([2, 3, 4, 5, 6, 7, 8])
max_depth = np.array([4, 5, 6, 7, 8, 9, 10])
min_leaf = np.array([1, 2, 3, 4, 5, 6, 7, 8])
indice = ['gini', 'entropy']

# Criação do dicionário
valores_grid = dict(min_samples_split=min_splits, max_depth=max_depth, min_samples_leaf=min_leaf, criterion=indice)

# Criar o modelo
tree = DecisionTreeClassifier()

# Aplicar os grids na GridSearch
tree_grid = GridSearchCV(estimator=tree, param_grid=valores_grid, cv=3)
tree_grid.fit(previsores, alvo)

# Melhores parametros
print(tree_grid.best_params_)

# Melhores resultados
print(tree_grid.best_score_)

{'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2}
0.8069038179934998


O desempenho recebeu uma melhora importante. Porém, agora vamos aplicar a técnica de __Bagging__.

Ao invés de realizar uma rodada com valores a serem testados, iremos construir a árvore com seus valores padrão novamente, porém iremos aplicar a técnica citada acima. Essa técnica irá criar árvores com dados diferentes.

In [8]:
from sklearn.ensemble import BaggingClassifier

In [11]:
# Criação do modelo
tree2 = DecisionTreeClassifier()
modelo_bagging = BaggingClassifier(estimator=tree2, n_estimators=100, max_samples=.5)
resultado = cross_val_score(modelo_bagging, previsores, alvo, cv=skfold)
print(resultado.mean())

0.8352512589735347


É importante ressaltar que diferente do que fizemos anteriormente, onde fizemos o ajuste dos parâmetros e exibimos os melhores parâmetros entrados, dessa vez utilizamos a árvore com seus valores padrão mas aplicamos a técnica de bagging, onde são criadas diversas árvores (100, no nosso caso) com dados diferentes para cada uma delas. Isso permite que o modelo aprenda diferentes padrões e seja mais generalizável.